<a href="https://colab.research.google.com/github/18K41A0565/NNDL/blob/main/Assignment_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Adagrad + SGD
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_excel("/content/drive/MyDrive/Load_Data_in_kW.xlsx",header=None)
data

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980
...,...
2179,4626.51840
2180,4567.90752
2181,4723.63164
2182,5150.27520


In [6]:
df = data
q = df[0].quantile(0.99)
df[df[0] < q]
q_low = df[0].quantile(0.01)
q_hi  = df[0].quantile(0.99)

df_filter= df[(df[0] < q_hi) & (df[0] > q_low)]
df_filter

,0
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980
...,...
2179,4626.51840
2180,4567.90752
2181,4723.63164
2182,5150.27520


In [7]:
print(len(data))
print(len(df_filter))
if(len(data) == len(df_filter)):
    print('No outliners')

2184
2140


In [8]:
#normalize
data = df_filter
data_n = data.copy()
data_n = (data_n - data_n.min())/(data_n.max() - data_n.min())
data_n

,0
0,0.372950
1,0.254941
2,0.235273
3,0.273574
4,0.299162
...,...
2179,0.180927
2180,0.168764
2181,0.201080
2182,0.289619


In [9]:
train_data,test_data = train_test_split(data_n,test_size=0.1)
print(len(train_data))
train_data

1926


,0
1720,0.154401
1183,0.310355
235,0.782842
792,0.343965
1946,0.387798
...,...
140,0.603015
1268,0.401514
1847,0.594345
1416,0.659431


In [10]:
xar = []
yar = []
for i in range(len(train_data.iloc[:,0])-(7*24)):
    xar.append(train_data.iloc[i,0])
    yar.append(train_data.iloc[i+(7*24),0])

In [11]:
from sympy import *
m, c = symbols('m c')
exp = '0.5*(y-m*x-c)**2'
exp_diff = Derivative(exp,m)
dfm = exp_diff.doit()
exp_diff = Derivative(exp,c)
dfc = exp_diff.doit()
err=[]
print(f'1st order derivative w.r.t m and c :\nde/dm = {dfm} \nde/dc = {dfc}')
#step 1:
m1=1
c1=-1
learning_rate=0.1
itr=20
gm=gc=0
e=10**(-8)

for i in range(0,itr):
  er=0
  for j in range(0,len(xar)):
    er +=  ((yar[j] - (m1 * xar[j]) - c1))**2
    #step 2:
    m,c,x,y = symbols('m c x y')
    dedm = dfm.subs(m,m1)
    dedm = dedm.subs(c, c1) 
    dedm= dedm.subs(x, xar[j]) 
    dedm = dedm.subs(y, yar[j])
    dedm = round(dedm,2)
    dedc = dfc.subs(c,c1)
    dedc = dedc.subs(m, m1) 
    dedc = dedc.subs(x, xar[j]) 
    dedc = dedc.subs(y, yar[j])
    dedc = round(dedc,2)
    gm=gm+(dedm**2)
    gc=gc+(dedc**2)
    #step 3:
    vm = (-1.0)*(learning_rate/(gm+e)**(0.5))*dedm
    vc = (-1.0)*(learning_rate/(gc+e)**(0.5))*dedc
    #step 4:
    m1 = m1+vm
    m1 = round(m1,2)
    c1 = c1+vc
    c1 = round(c1,2)
    er = (1/(2*len(xar)))*(er)
    err.append(er)

print(f'min value of m = {m1} \nmin value of c = {c1}')

1st order derivative w.r.t m and c :
de/dm = -1.0*x*(-c - m*x + y) 
de/dc = 1.0*c + 1.0*m*x - 1.0*y
min value of m = 0.28 
min value of c = 0.25


In [12]:
#RMSE - Training data
train_data_pred = [i for i in xar]
for k in range(len(xar)):
    train_data_pred[k]=(m1*xar[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,train_data_pred)
mse

0.056307537734275735

In [13]:
#MSE - Test Data
xar = []
yar = []
for i in range(len(test_data.iloc[:,0])-24):
    xar.append(test_data.iloc[i,0])
    yar.append(test_data.iloc[i+24,0])
test_data_pred = [i for i in xar]
for k in range(len(xar)):
    test_data_pred[k]=(m1*xar[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(yar,test_data_pred)
mse

0.06284609627579407